In [24]:
from scipy.stats import norm
alpha = 0.1

norm.ppf(0.5*alpha, loc=0, scale=1)
#qnorm(0.5*alpha, mean = 0, sd = 1, lower.tail = FALSE, log.p = FALSE)

-1.6448536269514729

In [1]:
%pylab inline
import logging
import numpy as np
import pandas as pd
import xgboost as xgb
from uplift.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

import logging
import pickle
from datetime import timedelta
from os.path import join as pjoin
import random


log_format = '[%(asctime)s] %(name)-25s %(levelname)-8s %(message)s'
logging.basicConfig(
    format=log_format,
    level=logging.INFO,
)
logger = logging.getLogger(__name__)


Populating the interactive namespace from numpy and matplotlib


In [5]:
# Чтение данных
df_clients = pd.read_csv('data/clients.csv', index_col='client_id')
df_train = pd.read_csv('data/uplift_train.csv', index_col='client_id')
df_test = pd.read_csv('data/uplift_test.csv', index_col='client_id')

# Извлечение признаков

df_clients['first_issue_time'] = \
    (pd.to_datetime(df_clients['first_issue_date'])
     - pd.Timestamp('1970-01-01')) // pd.Timedelta('1s')
df_clients['first_redeem_time'] = \
    (pd.to_datetime(df_clients['first_redeem_date'])
     - pd.Timestamp('1970-01-01')) // pd.Timedelta('1s')
df_clients['issue_redeem_delay'] = df_clients['first_redeem_time'] \
    - df_clients['first_issue_time']
df_clients = df_clients.drop(['first_issue_date', 'first_redeem_date'], axis=1)

df_features = pd.DataFrame({
    'gender_M': (df_clients['gender'] == 'M').astype(int),
    'gender_F': (df_clients['gender'] == 'F').astype(int),
    'gender_U': (df_clients['gender'] == 'U').astype(int),
    'age': df_clients['age'],
    'first_issue_time': df_clients['first_issue_time'],
    'first_redeem_time': df_clients['first_redeem_time'],
    'issue_redeem_delay': df_clients['issue_redeem_delay'],
}).fillna(0)

indices_train = df_train.index
indices_learn, indices_valid = train_test_split(df_train.index, test_size=0.3, random_state=123)

X_learn = df_features.loc[indices_train, :]
target_learn = df_train.loc[indices_train, 'target']
treatment_learn = df_train.loc[indices_train, 'treatment_flg']


In [3]:
logger.info('Loading features...')
with open('features.pkl', 'rb') as f:
    features: pd.DataFrame = pickle.load(f)
logger.info('Features are loaded')

logger.info(f'Features shape: {features.shape}')

logger.info('Preparing data sets...')
features.set_index('client_id', inplace=True)


X_train = features.loc[indices_train, :]
X_train = X_train.astype('float32')
X_train = X_train.replace([np.inf, -np.inf], np.nan).fillna(0)

[2020-11-07 17:30:44,867] __main__                  INFO     Loading features...
[2020-11-07 17:30:57,510] __main__                  INFO     Features are loaded
[2020-11-07 17:30:57,511] __main__                  INFO     Features shape: (400162, 334)
[2020-11-07 17:30:57,512] __main__                  INFO     Preparing data sets...


In [4]:
print(X_train.shape)

(200035, 333)
